# IMPORT LIB

In [1]:
import gc
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_error

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import Bidirectional, LSTM
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.layers import Concatenate, Add, GRU


2021-10-31 22:56:45.186761: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2021-10-31 22:56:45.186900: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
#Modify according to the seeds you set
np.random.seed(42)
tf.random.set_seed(42)

# Data Load

In [3]:
train_df = pd.read_csv('../input/ventilator-pressure-prediction/train.csv')
print(f"train_df: {train_df.shape}")
train_df.head()

test_df = pd.read_csv('../input/ventilator-pressure-prediction/test.csv')
print(f"test_df: {test_df.shape}")
test_df.head()

train_df: (6036000, 8)
test_df: (4024000, 7)


,id,breath_id,R,C,time_step,u_in,u_out
0,1,0,5,20,0.000000,0.000000,0
1,2,0,5,20,0.031904,7.515046,0
2,3,0,5,20,0.063827,14.651675,0
3,4,0,5,20,0.095751,21.230610,0
4,5,0,5,20,0.127644,26.320956,0


In [4]:
all_pressure = sorted(train_df.pressure.unique())
PRESSURE_MIN = all_pressure[0].item()
PRESSURE_MAX = all_pressure[-1].item()
PRESSURE_STEP = ( all_pressure[1] - all_pressure[0] ).item()

# Feature Engineering

In [5]:
def prepare_set(df): #CV 0.1579
    
    np.random.seed(42)
    df['noise'] = np.random.choice([0, PRESSURE_STEP, -PRESSURE_STEP], len(df))
    
    df['flow'] = np.sqrt(2*df['u_in'])
    
    df['u_in_log'] = np.log(df['u_in']).replace(-np.inf,0)
        
    #df['area'] = df['time_step'] * df['u_in']
    #df['area'] = df.groupby('breath_id')['area'].cumsum()
    df['state'] = np.array([1 if x>0 else 0 for x in df['u_in']]) - df['u_out']
    
    #df['exhale'] = df.groupby('breath_id')['u_out'].cumsum()
        
    df['delta_time'] = df.groupby('breath_id')['time_step'].diff().fillna(0)
    df['delta_u_in'] = df.groupby(df['breath_id'])['u_in'].diff().fillna(0).reset_index(level=0,drop=True)     
    df['delta_flow'] = df.groupby(df['breath_id'])['flow'].diff().fillna(0).reset_index(level=0,drop=True)          
    
    df['inhale_time'] = df['state'] *  df['delta_time'] * (1 - df['u_out'])
    df['inhale_time'] = (df.groupby(df['breath_id'])['inhale_time']).cumsum()  * (1 - df['u_out'])

    df['flow_1st_der'] = (df['delta_flow'] /df['delta_time']).fillna(0)
    df['u_in_cumsum'] = (df['u_in']).groupby(df['breath_id']).cumsum()
    df['flow_cumsum'] = (df['flow']).groupby(df['breath_id']).cumsum()
    
    df['u_in_lag2'] = df.groupby('breath_id')['u_in'].shift(2).fillna(0).reset_index(level=0,drop=True)
    df['u_in_lag4'] = df.groupby('breath_id')['u_in'].shift(4).fillna(0).reset_index(level=0,drop=True)

    df['u_in_lag-1'] = df.groupby('breath_id')['u_in'].shift(-1).fillna(0).reset_index(level=0,drop=True)
    
    df['u_in_dif2'] = df['u_in'] - df['u_in_lag2']
    df['u_in_dif4'] = df['u_in'] - df['u_in_lag4']
    df['u_in_dif-1'] = df['u_in'] - df['u_in_lag-1']
    
    df['volume_mean']= df['flow'] * df['delta_time']

       
    df['volume_in_cumsum']=df.groupby('breath_id')['volume_mean'].cumsum()     
    df['volume_in_cumsum_reverse']=df.groupby(df['breath_id'])['volume_in_cumsum'].transform('max')  - df['volume_in_cumsum']
        
    df['_volume'] = df['volume_in_cumsum'] * (1 - df['u_out'])
    df['tidal_volume']=df.groupby(df['breath_id'])['_volume'].transform('max')
    
    df['volume_part'] = (df['volume_mean']/df['volume_in_cumsum']).fillna(0)
    df['volume_part'] = df.groupby('breath_id')['volume_part'].shift(-1).fillna(0).reset_index(level=0,drop=True)
    
    df['time_constant'] = df.groupby(df['breath_id'])['inhale_time'].transform('max')
    df['V_dot'] = df['tidal_volume'] / df['time_constant']
    

    df['u_in_rol_q0.25'] = df.groupby(df['breath_id'])['u_in'].rolling(window=10, min_periods=1, center=True).quantile(0.25).reset_index(level=0,drop=True)
    df['u_in_rol_q0.75'] = df.groupby(df['breath_id'])['u_in'].rolling(window=10, min_periods=1, center=True).quantile(0.75).reset_index(level=0,drop=True)

    
    df['dP_on_R'] = df['flow'] * df['R']/1000 
    df['lung_expand'] = df['dP_on_R'] * df['C'] 
    df['dP_on_C'] = df['flow'] * df['delta_time'] * 1000 / df['C'] 
    df['dP_on_C_cumsum'] = df.groupby('breath_id')['dP_on_C'].cumsum()   
    
    df['RC'] = df['R'].astype(str) + df['C'].astype(str)
    df['R'] = df['R'].astype(str)
    df['C'] = df['C'].astype(str)
    df = pd.get_dummies(df)

    df['u_in_1st_order_grad'] = np.stack(df.groupby(df['breath_id'])['u_in'].apply(np.gradient).values).reshape(len(df),)
    

    df = df.fillna(0)
    
    return df

train = prepare_set(train_df)
test = prepare_set(test_df)

del train_df, test_df
gc.collect()

/opt/conda/lib/python3.7/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


0

In [6]:
targets = train[['pressure']].to_numpy().reshape(-1, 80)
u_outs = train[['u_out']].to_numpy().reshape(-1, 80)

train.drop(['pressure', 'id', 'breath_id'], axis = 1, inplace = True)
test = test.drop(['id', 'breath_id'], axis = 1)

print(f"train: {train.shape}")

train: (6036000, 50)


In [7]:
scaler = RobustScaler()
train = scaler.fit_transform(train)
test = scaler.transform(test)

train = train.reshape(-1, 80, train.shape[-1])
test = test.reshape(-1, 80, train.shape[-1])

print(f"train: {train.shape} \n targets: {targets.shape}")

train: (75450, 80, 50) 
 targets: (75450, 80)


In [8]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
    BATCH_SIZE = strategy.num_replicas_in_sync * 64
    print("Running on TPU:", tpu.master())
    print(f"Batch Size: {BATCH_SIZE}")
    
except ValueError:
    strategy = tf.distribute.get_strategy()
    BATCH_SIZE = 512
    print(f"Running on {strategy.num_replicas_in_sync} replicas")
    print(f"Batch Size: {BATCH_SIZE}")

2021-10-31 22:59:36.882374: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-10-31 22:59:36.885750: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2021-10-31 22:59:36.885791: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-10-31 22:59:36.885829: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ad1d2093a456): /proc/driver/nvidia/version does not exist
2021-10-31 22:59:36.888932: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operation

Running on TPU: grpc://10.0.0.2:8470
Batch Size: 512


In [9]:
def GBVPP_loss(y_true, y_pred, cols = 80):
    u_out = y_true[:, cols: ]
    y = y_true[:, :cols ]

    w = 1 - u_out
    mae = w * tf.abs(y - y_pred)
    return tf.reduce_sum(mae, axis=-1) / tf.reduce_sum(w, axis=-1)
   
def get_model():  
    x_input = keras.Input(shape=(train.shape[-2:]))
    
    x1 = layers.Bidirectional(layers.LSTM(units=768, return_sequences=True))(x_input)
    x2 = layers.Bidirectional(layers.LSTM(units=512, return_sequences=True))(x1)
    x3 = layers.Bidirectional(layers.LSTM(units=384, return_sequences=True))(x2)
    x4 = layers.Bidirectional(layers.LSTM(units=256, return_sequences=True))(x3)
    x5 = layers.Bidirectional(layers.LSTM(units=128, return_sequences=True))(x4)
    
    z2 = layers.Bidirectional(layers.GRU(units=384, return_sequences=True))(x2)
    
    z31 = layers.Multiply()([x3, z2])
    z31 = layers.BatchNormalization()(z31)
    z3 = layers.Bidirectional(layers.GRU(units=256, return_sequences=True))(z31)
    
    z41 = layers.Multiply()([x4, z3])
    z41 = layers.BatchNormalization()(z41)
    z4 = layers.Bidirectional(layers.GRU(units=128, return_sequences=True))(z41)
    
    z51 = layers.Multiply()([x5, z4])
    z51 = layers.BatchNormalization()(z51)
    z5 = layers.Bidirectional(layers.GRU(units=64, return_sequences=True))(z51)
    
    x = layers.Concatenate(axis=2)([x5, z2, z3, z4, z5])
    
    x = layers.Dense(units=128, activation='selu')(x)
    
    x_output = layers.Dense(units=1)(x)
    
    model = keras.Model(inputs=x_input, outputs=x_output)
    
    model.compile(optimizer = "adam", 
                  #loss = "mae",
                  loss=GBVPP_loss,
                 #sample_weight_mode="temporal",
                 )
    
    return model  

In [10]:
gc.collect()

42

In [11]:
#keras.backend.clear_session()

In [12]:
train_df = pd.read_csv('../input/ventilator-pressure-prediction/train.csv')
train_preds = train_df[['id', 'breath_id', 'pressure']]
train_preds.loc[:, 'modified_breath_id'] = [i for i in range(len(train)) for _ in range(80)]

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


In [13]:
NUM_FOLD = 10
EPOCH = 40
BATCH_SIZE = 256


with strategy.scope():
    
    VERBOSE = 0
    test_preds = []
    
    kf = KFold(n_splits=NUM_FOLD, shuffle=True, random_state=100)
    
    for fold, (train_idx, test_idx) in enumerate(kf.split(train, targets)):
        X_train, X_valid = train[train_idx], train[test_idx]
        y_train, y_valid = targets[train_idx], targets[test_idx]
        u_out_train, u_out_valid = u_outs[train_idx], u_outs[test_idx]  
        
        model = get_model()
        model_path = f'../input/gb-vpp-yet-another-lstm-colab/best_valid_fold_{fold+1}.hdf5' 
        model.load_weights(model_path)
        
        #keras.backend.set_value(model.optimizer.lr, 0.00007)
        model.compile(optimizer = tf.keras.optimizers.SGD(learning_rate=0.0007), loss=GBVPP_loss)
        
        plateau = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.7, patience=3, verbose=1, min_lr=1e-08)
        estop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min',restore_best_weights=False)

        checkpoint_filepath = f"best_valid_fold_{fold+1}.hdf5"
        sv = keras.callbacks.ModelCheckpoint(
            checkpoint_filepath, monitor='val_loss', verbose=1, save_best_only=True,
            save_weights_only=False, mode='auto', save_freq='epoch',
            options=None
        )
        
        
        model.fit(X_train, np.append(y_train, u_out_train, axis =1),
                  validation_data = (X_valid, np.append(y_valid, u_out_valid, axis =1)), epochs = EPOCH, 
                  batch_size = BATCH_SIZE, callbacks = [estop, plateau, sv],
                  shuffle=True,
                 )
        
        model.save(f"end_of_fold_{fold+1}.hdf5")
            
        y_true = y_valid.squeeze().reshape(-1, 1)
        y_pred = model.predict(X_valid, batch_size=BATCH_SIZE).squeeze().reshape(-1, 1)
        score = mean_absolute_error(y_true, y_pred)
        train_preds.loc[train_preds.loc[:, 'modified_breath_id'].isin(test_idx), 'pressure'] = y_pred
        print(f"Fold-{fold+1} | OOF Score: {score}")
        
        test_preds.append(model.predict(test, batch_size=BATCH_SIZE).squeeze().reshape(-1, 1).squeeze())
        
        gc.collect()

2021-10-31 23:00:21.828149: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1086480000 exceeds 10% of free system memory.


Epoch 1/40
266/266 [==============================] - 81s 199ms/step - loss: 0.1450 - val_loss: 0.2217

Epoch 00001: val_loss improved from inf to 0.22174, saving model to best_valid_fold_1.hdf5
Epoch 2/40
266/266 [==============================] - 36s 135ms/step - loss: 0.1500 - val_loss: 0.2347

Epoch 00002: val_loss did not improve from 0.22174
Epoch 3/40
266/266 [==============================] - 36s 135ms/step - loss: 0.1474 - val_loss: 0.2184

Epoch 00003: val_loss improved from 0.22174 to 0.21839, saving model to best_valid_fold_1.hdf5
Epoch 4/40
266/266 [==============================] - 36s 135ms/step - loss: 0.1479 - val_loss: 0.2452

Epoch 00004: val_loss did not improve from 0.21839
Epoch 5/40
266/266 [==============================] - 36s 135ms/step - loss: 0.1476 - val_loss: 0.1985

Epoch 00005: val_loss improved from 0.21839 to 0.19852, saving model to best_valid_fold_1.hdf5
Epoch 6/40
266/266 [==============================] - 36s 135ms/step - loss: 0.1444 - val_loss: 0

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


Fold-1 | OOF Score: 7.193707065321711


2021-10-31 23:25:32.089062: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 804800000 exceeds 10% of free system memory.
2021-10-31 23:26:02.733839: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1086480000 exceeds 10% of free system memory.


...

Fold-8 | OOF Score: 6.8821359510187055
Epoch 1/40
266/266 [==============================] - 84s 199ms/step - loss: 0.1524 - val_loss: 0.2366

Epoch 00001: val_loss improved from inf to 0.23660, saving model to best_valid_fold_9.hdf5
Epoch 2/40
266/266 [==============================] - 36s 135ms/step - loss: 0.1572 - val_loss: 0.2202

Epoch 00002: val_loss improved from 0.23660 to 0.22015, saving model to best_valid_fold_9.hdf5
Epoch 3/40
266/266 [==============================] - 36s 135ms/step - loss: 0.1564 - val_loss: 0.2276

Epoch 00003: val_loss did not improve from 0.22015
Epoch 4/40
266/266 [==============================] - 36s 135ms/step - loss: 0.1554 - val_loss: 0.2573

Epoch 00004: val_loss did not improve from 0.22015
Epoch 5/40
266/266 [==============================] - 36s 135ms/step - loss: 0.1552 - val_loss: 0.2420

Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0004899999825283885.

Epoch 00005: val_loss did not improve from 0.22015
Epoch 6/40
266/266 [=

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


Fold-9 | OOF Score: 8.045731011693714
Epoch 1/40
266/266 [==============================] - 84s 201ms/step - loss: 0.1787 - val_loss: 0.2204

Epoch 00001: val_loss improved from inf to 0.22043, saving model to best_valid_fold_10.hdf5
Epoch 2/40
266/266 [==============================] - 36s 135ms/step - loss: 0.1826 - val_loss: 0.2344

Epoch 00002: val_loss did not improve from 0.22043
Epoch 3/40
266/266 [==============================] - 36s 135ms/step - loss: 0.1815 - val_loss: 0.2511

Epoch 00003: val_loss did not improve from 0.22043
Epoch 4/40
266/266 [==============================] - 36s 135ms/step - loss: 0.1805 - val_loss: 0.2619

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0004899999825283885.

Epoch 00004: val_loss did not improve from 0.22043
Epoch 5/40
266/266 [==============================] - 36s 135ms/step - loss: 0.1255 - val_loss: 0.1779

Epoch 00005: val_loss improved from 0.22043 to 0.17792, saving model to best_valid_fold_10.hdf5
Epoch 6/40
266/266 [

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


Fold-10 | OOF Score: 7.170678200161252


In [14]:
fea_names = ['id', 'pressure']
train_preds[fea_names].to_csv('oof.csv', index=False)

In [15]:
#sub = pd.read_csv('../input/pressure-speed-and-weights-ltsm/submission.csv')
#sub.to_csv('submission.csv', index=False) 

In [16]:
ss = pd.read_csv('../input/ventilator-pressure-prediction/sample_submission.csv')

ss['pressure'] = np.median(np.vstack(test_preds),axis=0)
ss["pressure"] =\
    np.round( (ss.pressure - PRESSURE_MIN)/PRESSURE_STEP ) * PRESSURE_STEP + PRESSURE_MIN
ss.pressure = np.clip(ss.pressure, PRESSURE_MIN, PRESSURE_MAX)
ss.to_csv('submission.csv', index=False)